In [ ]:
import redis
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from brand.timing import timespecs_to_timestamps, timevals_to_timestamps

In [ ]:
# connect to Redis
# (requires supervisor to be already running at 127.0.0.1:6379,
#  modify host/port below if needed)

r = redis.Redis(host='127.0.0.1')

In [ ]:
# define cerebusAdapter graph for test (replace with test graph)

GRAPH_PATH = "/home/mrigott/Projects/brand-modules/brand-nsp/tests/test_cerebus_adapt/test_cerebus_adapt.yaml"

In [ ]:
# flush Redis database before test

r.flushdb()

In [ ]:
# start graph 
# (simulator/emit/etc. should already be running)

r.xadd('supervisor_ipstream', {
    'commands': 'startGraph',
    'file': GRAPH_PATH
})

In [ ]:
# stop graph 
# (wait a couple of seconds before running this block to get enough data)

r.xadd('supervisor_ipstream', {'commands': 'stopGraph'})

In [ ]:
# check that continuousNeural stream key exists

r.keys('*')

In [ ]:

# check that there are samples in the stream

streams = ['continuousNeural']
msg = 'Stream Info'
print(msg + '\n' + '-' * len(msg))
for stream in streams:
    n_entries = r.xlen(stream)
    if n_entries > 0:
        entry_dict = r.xrevrange(stream, count=1)[0][1]
        has_sync = True if b'sync' in entry_dict else False
        if has_sync:
            entry_dict
    else:
        has_sync = False

    row = f'{stream :24s}: {n_entries :6d}'
    if has_sync:
        row += f"\tsync={json.loads(entry_dict[b'sync'])}"
    else:
        row += '\tsync=None'
    print(row)

In [ ]:
# utility function

def scalarfrombuffer(*args, **kwargs):
    return np.frombuffer(*args, **kwargs)[0]

samp_per_packet = 30

# build dataframe with neural data

stream = 'continuousNeural'
entries = r.xrange(stream)
data = [None] * len(entries)
for i, (_, entry_data) in enumerate(entries):
    data[i] = {
        f: entry_data[f.encode()]
        for f in ['timestamps', 'BRANDS_time']
    }
    brands_time = (
        np.array(timespecs_to_timestamps(entry_data[b'BRANDS_time'])) *
        1e9).astype(np.uint64)
    udp_time = (np.array(timevals_to_timestamps(entry_data[b'udp_recv_time'])) *
                1e9).astype(np.uint64)
    nsp_idx = np.frombuffer(entry_data[b'timestamps'], dtype=np.uint32)
    samples = np.frombuffer(entry_data[b'samples'], dtype=np.int16).reshape((-1,samp_per_packet)).T
    data[i] = {
        'ts': brands_time[-1],
        'sync': nsp_idx[0],
        'udp_in': udp_time[-1],
        'samples': samples,
        'timestamps': nsp_idx,
    }
df = pd.DataFrame(data)
print(df)

# print ISI for cerebusAdapter data

isi = df['ts'].diff()[1:].values / 1e6
print(f'cerebusAdapter ISI: {isi.mean()} +- {isi.std()}'
      f' ({isi.min()} - {isi.max()})')

In [ ]:
# stack stream entries to build array with all continuous data

cont_data = np.vstack(df.samples.values).T
print(cont_data.shape)
print(cont_data)

In [ ]:
# plot continuous data

n_channels = 30
trange = slice(0, 30 * 100000)

fig, axes = plt.subplots(ncols=1,
                         nrows=n_channels,
                         figsize=(10, n_channels * 2),
                         sharey=False,
                         sharex=True)
for ich in range(n_channels):
    ax = axes.flat[ich]
    ax.plot(cont_data[ich, trange])
    ax.set_title(f'Ch {ich}')